# Taxonomic Annotation of Reads

This notebook will go through the workflow assigning taxonomy to reads from a microbiome

1. Taxonomic assignment of reads using Kraken2
2. Refinement of the taxonomic annotation using Bracken


## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [ ]:
# set the variables for your netid and xfile
netid = "MY_NETID"
xfile = "MY_XFILE"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/11_taxonomy"
%cd $work_dir

## Creating a config file
Let's create a config file with all of the variables we will need in the scripts below. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

In [ ]:
# create a config file with all of the variables you need
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export WORK_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/11_taxonomy" >> config.sh
!echo "export XFILE_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/05_getting_data" >> config.sh
!echo "export FASTQ_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/07_contam_removal" >> config.sh

In [ ]:
# check the config file to be sure it is correct
# Is your netid and xfile correct? Do you have the right directories?
!cat config.sh

## Step 1: Assigning taxonomy to the reads

In this step, we will try to assign taxonomy to each of the reads in our microbiome. 

In [ ]:
# Create a script to run bowtie2 to align reads to a human reference
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. bowtie2 runs on each of the fastq files in the trimmed $FASTQ_DIR
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for more time (24 hours)
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=24:00:00   
#SBATCH --partition=standard
#SBATCH --account=bh_class
#SBATCH --array=0-4                         
#SBATCH --output=Job-rem_human-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G                                    

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

PAIR1=${FASTQ_DIR}/${SAMPLE_ID}_1.fastq.gz
PAIR2=${FASTQ_DIR}/${SAMPLE_ID}_2.fastq.gz

### reads with human removed
BOWTIE_NAME="${WORK_DIR}/${SAMPLE_ID}_%.fastq.gz"
SAM_NAME="${WORK_DIR}/${SAMPLE_ID}_human_removed.sam"

### reads mapped to human
MET_NAME="${WORK_DIR}/${SAMPLE_ID}_hostmap.log"

apptainer run /contrib/singularity/shared/bhurwitz/bowtie2:2.5.1--py39h6fed5c7_2.sif bowtie2 \
    -p 24 -x $HUM_DB -1 $PAIR1 -2 $PAIR2 --un-conc-gz $BOWTIE_NAME 1> $SAM_NAME 2> $MET_NAME

rm $SAM_NAME
'''

with open('remove_human_parallel.sh', mode='w') as file:
    file.write(my_code)


#!/bin/bash -l
#SBATCH --job-name=kraken2_%a
#SBATCH --output=errout/kraken2_output_%a.txt
#SBATCH --error=errout/kraken2_error_%a.txt
#SBATCH --time=24:00:00
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=4G
#SBATCH --ntasks=1

# source configuration and copy to the result dir 
source $SLURM_SUBMIT_DIR/config/config_testing.sh
RESULT_DIR="$OUT_DIR/out_kraken2"
mkdir $RESULT_DIR
HUMAN_READ_DIR="$RESULT_DIR/human_reads"
NONHUMAN_READ_DIR="$RESULT_DIR/nonhuman_reads"
mkdir $HUMAN
mkdir $NONHUMAN

cp $SLURM_SUBMIT_DIR/config/config_testing.sh $RESULT_DIR

# echo for log
echo "job started"; hostname; date

# go to folder
cd $IN_DIR

# get sample ID
export SAMPLE_ID=`awk NR==$SLURM_ARRAY_TASK_ID $IN_LIST`

#debugging
echo sample list:  $SAMPLE_ID
echo slurm_id: $SLURM_ARRAY_TASK_ID

PAIR1=${SAMPLE_ID}_human_removed_1.fastq*
PAIR2=${SAMPLE_ID}_human_removed_2.fastq*

P1="$IN_DIR/$PAIR1"
P2="$IN_DIR/$PAIR2"

OUT="$RESULT_DIR/$SAMPLE_ID"
mkdir $OUT
DB="$DB_DIR"
# DB="$DB_DIR/database${KMER_SIZE}mers.kmer_distrib" TESTING TAXO USAGE

apptainer run $KRAKEN2 kraken2 --db ${DB} --paired --classified-out $OUT/cseqs.fq --output $OUT/kraken_results.txt --report $OUT/kraken_report.txt --use-n
ames --threads ${SLURM_CPUS_PER_TASK} $P1 $P2

#apptainer run $KRAKEN2 kraken2 \
#              --db ${DB} \ 
#              --paired \
#              --classified-out $OUT/cseqs.fq \
#              --output $OUT/kraken_results.txt \
#              --report $OUT/kraken_report.txt \
#              --use-names \
#              --threads ${SLURM_CPUS_PER_TASK} \ 
#              $P1 $P2 

# run krakentools to extract human/microbial reads
conda activate kraken2
# selects all reads from a given set of Kraken taxids (and all children)
# select all reads from human (9606 for human)
TAXID=9606
$KRAKENTOOLS/extract_kraken_reads.py \
             -k $OUT/kraken_results.txt \ 
             --report $OUT/kraken_report.txt \
             -s1 $P1 \
             -s2 $P2 \
             --taxid $TAXID \  
             --output $HUMAN_READ_DIR/r1.fq \  
             --output2 $HUMAN_READ_DIR/r2.fq \
             --include-children \
             --fastq-output

gzip $HUMAN_READ_DIR/r1.fq
gzip $HUMAN_READ_DIR/r2.fq

# selects all reads NOT from a given set of Kraken taxids (and all children)
# NOT human 
$KRAKENTOOLS/extract_kraken_reads.py \
             -k $OUT/kraken_results.txt \
             --report $OUT/kraken_report.txt \
             -s1 $P1 \
             -s2 $P2 \
             --taxid $TAXID \
             --output $NONHUMAN_READ_DIR/r1.fq \
             --output2 $NONHUMAN_READ_DIR/r2.fq \
             --include-children \
             --exclude \
             --fastq-output

gzip $NONHUMAN_READ_DIR/r1.fq
gzip $NONHUMAN_READ_DIR/r2.fq
 
echo "Finished `date`"

In [ ]:
# Check the code and make sure your script above was created.
!cat remove_human_parallel.sh

In [ ]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the remove_human_parallel.sh script?
!pwd
!ls

In [ ]:
# Let's run sbatch to run bowtie on each of your trimmed fastq files to remove human
# Remember that this may take a while to run, so take a break, and get a coffee.
!sbatch ./remove_human_parallel.sh

In [ ]:
# You can check if it is running using the squeue command
# Check for all jobs under your netid
!squeue --user=$netid

In [ ]:
# Once your jobs have run (or are running) you can check the progress
# and also look for errors in the *out files
# For example, you can look at Job-rem_human-0.out
!ls
!cat Job-rem_human-0.out

In [ ]:
# Double check that all of your files have run through the human screening
# Do you see a *_1.fastq.gz and *_2.fastq.gz file in the working directory?
# Of not, you will need to check your job out files above for clues about what went wrong.
!ls $work_dir

In [ ]:
# Do the fastq files (post-human filter) look smaller than the ones that were trimmed?
# list the file sizes for fastq files post-human filtering
!ls -l $work_dir

In [ ]:
# list the file sizes for fastq files before human filtering'
!ls -l /xdisk/bhurwitz/bh_class/$netid/assignments/06_qc_trimming/trimmed_reads

Great job! It looks like you have removed *most* of the human reads from your fastq files. We will double check this when we run kraken2 on the files to classify each of the reads by taxonomy.

Another quick note, you may need to remove additional contamination using the same approach. For example, the sequencing center may have use PhiX as a "spike-in" to assess the quality of the sequencing run with a known quantity of DNA. You can use the same approach as above to remove reads from any genome you think may be contaminating your sample.

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/07_contam_removal.ipynb $work_dir